### Deep Q-Learning

This method can keep the Segway balanced under enough training times, but it cannot control its motion well.

In [1]:
import distutils.util
import subprocess
#if subprocess.run('nvidia-smi').returncode:
#  raise RuntimeError(
#      'Cannot communicate with GPU. '
#      'Make sure you are using a GPU Colab runtime. '
#      'Go to the Runtime menu and select Choose runtime type.')

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

import mujoco

#@title Other imports and helper functions
import numpy as np

# Graphics and plotting.
import mediapy as media

# Import Renderer Class
import os
import sys

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from renderer import Renderer

import platform
print (platform.architecture())

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl
('64bit', 'ELF')


Load Model

In [2]:
model = mujoco.MjModel.from_xml_path('modelv2.xml')
data = mujoco.MjData(model)
renderer = Renderer(model)

Example Video:

In [3]:
DURATION  = 1   # seconds
FRAMERATE = 60  # Hz
import random

# Make a new camera, move it to a closer distance.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 12

frames = []
while data.time < DURATION:
  # Step the simulation.
  mujoco.mj_step(model, data)

  # Render and save frames.
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data, camera)
    pixels = renderer.render()
    frames.append(pixels.copy())

# Display video.
media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


Hyperparameter

In [4]:
import torch
from dqn import DQN

class Config:
    train_eps = 50000
    test_eps = 10
    epsilon_start = 0.95
    epsilon_end = 0.03
    epsilon_decay = 5000
    lr = 0.0001
    memory_capacity = 1000
    batch_size = 256
    target_update = 4
    hidden_dim = 128
    gamma = 0.998 
    coherent_movement = 6
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_states = 12 # Dim. Observation Space
    n_actions = 2 # Dim. Action Space

cfg = Config()
print(f"states: {cfg.n_states}, actions: {cfg.n_actions}")
agent = DQN(cfg.n_states, cfg.n_actions, cfg)  # create agents

states: 12, actions: 2


helper functions, similar to PID control

In [5]:
def get_state(old_coordinate, new_coordinate, tar_point):
    refpoint1 = new_coordinate[0]
    refpoint2 = new_coordinate[1]
    refpoint3 = new_coordinate[2]
    refpoint4 = new_coordinate[3]
    old_refpoint1 = old_coordinate[0]
    old_refpoint2 = old_coordinate[1]
    old_refpoint3 = old_coordinate[2]
    old_refpoint4 = old_coordinate[3]
    
    old_vector_pole = old_refpoint2 - old_refpoint1
    old_mid_point = (old_refpoint3 + old_refpoint4) / 2
    
    new_vector_pole = refpoint2 - refpoint1
    new_mid_point = (refpoint3 + refpoint4) / 2
    
    dir_vector = refpoint1 - new_mid_point
    dir_vector_2d = np.array([dir_vector[0], dir_vector[1]], dtype='float32')
    tar_vector = tar_point - refpoint1
    tar_vector_2d = np.array([tar_vector[0], tar_vector[1]], dtype='float32')
    
    delta_vector_pole = new_vector_pole - old_vector_pole
    delta_mid_point = new_mid_point - old_mid_point
    delta_mid_point_2d = np.array([delta_mid_point[0], delta_mid_point[1]], dtype='float32')
    
    return np.float32(np.hstack((new_vector_pole, delta_vector_pole, delta_mid_point_2d, dir_vector_2d, tar_vector_2d)))

def get_coordinate(data):
    refpoint1 = np.array(data.geom_xpos[model.geom("refpoint1").id])
    refpoint2 = np.array(data.geom_xpos[model.geom("refpoint2").id])
    refpoint3 = np.array(data.geom_xpos[model.geom("refpoint3").id])
    refpoint4 = np.array(data.geom_xpos[model.geom("refpoint4").id])
    ans = np.array([refpoint1, refpoint2, refpoint3, refpoint4], np.float32)
    return ans

def toCtrl(action):
    return np.array([action / 5 - 0.1, action / 5 - 0.1], dtype='float64') * 1.5

def isDone(data, velocity, distence_to_tar):
    return data.geom_xpos[model.geom("refpoint2").id][2] < 0.19 # The episode stops if the Segway goes down.

def toVelocity(state):
    return getLength(state[6], state[7])
    
def toDistence(state):
    return getLength(state[10], state[11])

def getLength(x, y):
    return np.sqrt(x*x + y*y)

train

In [6]:
FRAMERATE = 60  # Hz
P_VIDEO = 200

In [ ]:
for i_ep in range(cfg.train_eps):
    ep_reward = 0  # Record rewards within a episode
    ep_step = 0
    mujoco.mj_resetData(model, data) # Reset the environment and return to the initial state
    mujoco.mj_step(model, data)
    old_coordinate = get_coordinate(data)
    mujoco.mj_step(model, data)
    new_coordinate = get_coordinate(data)
    tar_point = new_coordinate[0]
    state = get_state(old_coordinate, new_coordinate, tar_point)
    frames = []
    while True:
        ep_step += 1
        action = agent.choose_action(state) # choose action
        data.ctrl = toCtrl(action)
        for x in range(cfg.coherent_movement):
            if (i_ep + 1) % P_VIDEO == 0 and len(frames) < data.time * FRAMERATE:
                renderer.update_scene(data, camera)
                pixels = renderer.render()
                frames.append(pixels.copy())
            mujoco.mj_step(model, data)
        old_coordinate = new_coordinate
        new_coordinate = get_coordinate(data)
        next_state = get_state(old_coordinate, new_coordinate, tar_point)
        velocity = toVelocity(next_state)
        distence_to_tar = toDistence(next_state)
        done = isDone(data, velocity, distence_to_tar)
        reward = 1
        if done:
            reward = -100
        agent.memory.push(state, action, reward, next_state, done)  # 保存transition
        state = next_state  # update next state
        agent.update()  # update agent
        ep_reward += reward  # add rewards
        if done or ep_step == 2000:
            break
    if (i_ep + 1) % cfg.target_update == 0:  # Agent target network update
        agent.target_net.load_state_dict(agent.policy_net.state_dict())

    if (i_ep + 1) % P_VIDEO == 0:     
        media.show_video(frames, fps=FRAMERATE)
        print(f'Episode：{i_ep+1}/{cfg.train_eps}, Reward:{ep_reward:.2f}, Step:{ep_step:.2f} Epislon:{agent.epsilon(agent.frame_idx):.3f}')

This browser does not support the video tag.


Episode：200/50000, Reward:80.00, Step:181.00 Epislon:0.109


This browser does not support the video tag.


Episode：400/50000, Reward:162.00, Step:263.00 Epislon:0.030


This browser does not support the video tag.


Episode：600/50000, Reward:234.00, Step:335.00 Epislon:0.030


This browser does not support the video tag.


Episode：800/50000, Reward:187.00, Step:288.00 Epislon:0.030
